# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import (weather_api_key, g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacay_df = pd.read_csv('clean city data.csv', index_col=0)
vacay_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bluff,-46.6000,168.3333,63.55,69,12,11.21,NZ,1610502336
1,rikitea,-23.1203,-134.9692,77.99,78,3,16.73,PF,1610502336
2,san patricio,28.0170,-97.5169,44.60,70,90,4.61,US,1610502336
3,tuatapere,-46.1333,167.6833,75.22,43,0,3.51,NZ,1610502336
4,batagay,67.6333,134.6333,-43.56,71,1,1.63,RU,1610502337


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key = g_key)

In [4]:
# Heatmap: Use the Lat and Lng as locations and Humidity as the weight
location = vacay_df[['Lat', 'Lng']]
weight = vacay_df['Humidity']
vac = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=weight, dissipating=False, max_intensity=300, point_radius=5)
vac.add_layer(heat_layer)
vac

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
weather_condition_df = vacay_df.loc[(vacay_df['Max Temp'] < 80) & (vacay_df['Max Temp'] > 70) 
                                    & (vacay_df['Wind Speed'] < 10) 
                                    & (vacay_df['Cloudiness'] == 0)].dropna()
weather_condition_df                                      

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,chotila,22.4167,71.1833,77.00,79,0,7.90,IN,1610349787
38,vila velha,-20.3297,-40.2925,77.00,83,0,6.91,BR,1610349790
136,saldanha,-33.0117,17.9442,75.20,69,0,2.30,ZA,1610349613
185,ahmadpur east,29.1431,71.2598,73.87,18,0,8.95,PK,1610349835
186,djibo,14.1022,-1.6306,70.32,22,0,7.20,BF,1610349835
191,arlit,18.7369,7.3853,70.21,21,0,7.20,NE,1610349837
279,chuy,-33.6971,-53.4616,71.33,92,0,9.24,UY,1610349862
288,puerto ayora,-0.7393,-90.3518,72.00,92,0,4.90,EC,1610349864
476,khanpur,28.6453,70.6567,74.93,17,0,7.72,PK,1610349921


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = weather_condition_df[['City', 'Country', 'Lat', 'Lng']].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,chotila,IN,22.4167,71.1833,
38,vila velha,BR,-20.3297,-40.2925,
136,saldanha,ZA,-33.0117,17.9442,
185,ahmadpur east,PK,29.1431,71.2598,
186,djibo,BF,14.1022,-1.6306,
191,arlit,NE,18.7369,7.3853,
279,chuy,UY,-33.6971,-53.4616,
288,puerto ayora,EC,-0.7393,-90.3518,
476,khanpur,PK,28.6453,70.6567,


In [16]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
      # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    name_address = requests.get(base_url, params=params)
    # convert to json
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,chotila,IN,22.4167,71.1833,Hotel Sunshine Royal Stay
38,vila velha,BR,-20.3297,-40.2925,Hotel Vitória Palace
136,saldanha,ZA,-33.0117,17.9442,Blue Bay Lodge
185,ahmadpur east,PK,29.1431,71.2598,Irfan fabrics
186,djibo,BF,14.1022,-1.6306,Centre Des Ressources DPENA
191,arlit,NE,18.7369,7.3853,Case de Passage De la Prefecture
279,chuy,UY,-33.6971,-53.4616,Nuevo Hotel Plaza
288,puerto ayora,EC,-0.7393,-90.3518,Castillo Galapagos
476,khanpur,PK,28.6453,70.6567,Ahmad bartan store


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
vac.add_layer(marker_layer)

# Display figure
vac

Figure(layout=FigureLayout(height='420px'))